In [ ]:
import cv2
import time
import os
import errno
import traceback

CAM_INDEX = 0
DEV_PATH = "/dev/video0"          # adjust if different
RETRY_DELAY = 1.0                 # seconds between reopen attempts
MAX_CONSECUTIVE_FAILS = 5         # escalate after repeated read fails

def device_exists(path):
    return os.path.exists(path)

def try_open(index, backend=None):
    try:
        return cv2.VideoCapture(index) if backend is None else cv2.VideoCapture(index, backend)
    except Exception:
        return None

def safe_release(cap):
    try:
        if cap is not None:
            cap.release()
    except Exception:
        pass

def main_loop():
    cap = try_open(CAM_INDEX)
    if cap is None or not cap.isOpened():
        print("Initial open failed; will retry")
        safe_release(cap)
        cap = None

    consecutive_fails = 0

    try:
        while True:
            # If camera not opened, try reopen if device node exists
            if cap is None or not cap.isOpened():
                if device_exists(DEV_PATH):
                    print("Device present ? attempting to open camera")
                    cap = try_open(CAM_INDEX)
                    if cap and cap.isOpened():
                        print("Camera opened")
                        consecutive_fails = 0
                    else:
                        safe_release(cap)
                        cap = None
                        time.sleep(RETRY_DELAY)
                        continue
                else:
                    # device node missing: wait and keep checking
                    print("Device node missing ? waiting")
                    time.sleep(RETRY_DELAY)
                    continue

            # Read frame with timeout-protection
            try:
                ret, frame = cap.read()
            except Exception as e:
                # read itself raised (e.g., USB disconnect). Clean up and retry.
                print("Exception during read:", repr(e))
                traceback.print_exc()
                safe_release(cap)
                cap = None
                time.sleep(RETRY_DELAY)
                continue

            if not ret or frame is None:
                consecutive_fails += 1
                print(f"Frame read failed ({consecutive_fails})")
                if consecutive_fails >= MAX_CONSECUTIVE_FAILS:
                    print("Too many consecutive failures ? releasing and retrying")
                    safe_release(cap)
                    cap = None
                    consecutive_fails = 0
                    time.sleep(RETRY_DELAY)
                else:
                    # brief pause before next attempt
                    time.sleep(0.1)
                continue

            consecutive_fails = 0

            # Display (ensure GUI calls are made on the main thread)
            try:
                cv2.imshow("cam", frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            except Exception as e:
                # GUI failure (e.g., X server issue) ? release camera and stop
                print("GUI/display error:", repr(e))
                traceback.print_exc()
                break

    except KeyboardInterrupt:
        print("Interrupted by user")
    finally:
        safe_release(cap)
        try:
            cv2.destroyAllWindows()
        except Exception:
            pass

if __name__ == "__main__":
    main_loop()
